# Imports and Setup

In [1]:
import requests
import pandas as pd
import configparser
import zeit

import plotly.express as px

In [2]:
c = configparser.ConfigParser()
c.read("config.ini")
zeit_key = c.get("Zeit", "key")

#set the API, authenticate and check status
api = zeit.API()
api.set_token(zeit_key)
api.get_status()

'everything ok'

# call the API

In [12]:
#call to the API and facet by authors
headers = {"X-Authorization": zeit_key}
params = {"facet_field": "author", "limit": 2}
url = "http://api.zeit.de/content"
r = requests.get(url=url, headers = headers, params=params)

# Construct Series

In [13]:
#construct a list from the facet field
l = r.json()["facets"]["author"]

In [14]:
#convert the list to a pandas Series , quite intensive work
long_series = pd.Series(dtype = "object")
for i in range(0,len(l)-2, 2):
    author = l[i]
    count = l[i+1]
    long_series[author] = count

# read in again

In [15]:
long_series.to_json("../data/long_series.json")

In [17]:
long_series = pd.read_json("../data/long_series.json", typ = "series", )

In [18]:
#filter the list for better results
series = long_series[long_series > 10]
#show the top authors by their articles
series = series.sort_values(ascending=False)
series[:10]

dpa                  3199
AFP                  1840
redaktion            1549
Oliver Fritsch       1425
Reuters              1327
Christoph Drösser    1322
Katharina Schuler    1225
Theo Sommer          1224
Tina Groll           1206
Kai Biermann         1192
dtype: int64

# explore names 

In [ ]:
#pandas series for names
names = long_series.index.to_series(index = pd.RangeIndex(start = 0, stop = len(long_series))) #new series from the index of long series
names = names.str.split().explode(ignore_index = True)

#show the most used names
names = names.value_counts()

In [ ]:
names[:200]

In [ ]:
df = names[:150].to_frame(name = "count") #construct a dataframe from the series
df["gender"] = "m" #set the gender as m
df.to_csv("../data/names.csv") #save to csv

#now i cleaned everything

In [3]:
df = pd.read_csv("../data/names.csv", names = ["names", "count", "gender"], header = 0) #read in the data again
df.head()

,names,count,gender
0,Peter,625,m
1,Hans,564,m
2,Michael,534,m
3,Thomas,429,m
4,Wolfgang,416,m


In [77]:
fig = px.bar(df[:60], y = "count", color="gender", text = "names",
    color_discrete_sequence = ["#efba32", "#c12e35"], labels = {"m": "männlicher Vorname", "f": "weiblicher Vorname"},
)

In [76]:
fig.update_layout(
        paper_bgcolor = "#f4f4f4",
        font = {"color":"#181818", "family" : "futura", "size": 13 },
        showlegend = True,
        plot_bgcolor = "#f4f4f4",
        margin = {"t": 0, "b": 0, "l": 0, "r": 0},
        hovermode = "x", 
        hoverlabel = {"bgcolor": "white", "bordercolor" : "white", "font_color": "black"},
        legend = dict(
            bgcolor = "rgba(0,0,0,0)",
            title = "Geschlecht",
            x = 0.9, y = 1,
            bordercolor = "rgba(0,0,0,0)",
            borderwidth = 0,
        ),)


fig.update_yaxes(ticklabelposition="outside top", title = None,
        gridcolor = "lightgrey", nticks = 4)

fig.update_xaxes(
    rangeslider_visible=True, range = (-1,10),
    ticklabelposition="outside top",
    showspikes = True, spikecolor = "grey", spikethickness = 2, spikemode = "across", 
    spikedash = "solid",
    linecolor = "lightgray",
    showgrid = False,
    title = "Bereich einstellen",
    )
fig.update_traces(
    hovertemplate = "<b>%{y} %{text}'s</b> <br>Platz: %{x} ",
)

# explore top authors

In [19]:
series[:20]

dpa                  3199
AFP                  1840
redaktion            1549
Oliver Fritsch       1425
Reuters              1327
Christoph Drösser    1322
Katharina Schuler    1225
Theo Sommer          1224
Tina Groll           1206
Kai Biermann         1192
Tilman Steffen       1174
Josef Joffe          1137
Michael Thumann      1089
Patrick Beuth        1048
Mark Schieritz       1027
Lisa Caspari         1000
Christian Spiller     997
Gunter Hofmann        932
Gero von Randow       909
Mark Spörrle          873
dtype: int64

In [ ]:
api.search_for("Barbara", "author")

In [ ]:
author_list = ["dpa", "AFP","redaktion", "Oliver-Fritsch","Reuters", "Christoph-Drösser", 
"Katharina-Schuler", "Theo-Sommer", "Tina-Groll", "Kai-Biermann", "Tilman-Steffen",
"Josef-Joffe","Michael-Thumann", "Patrick-Beuth",  "Mark-Schieritz", "Lisa-Caspari"
"Christian-Spiller", "Gunter-Hofmann", "Gero-von-Randow", "Mark-Spörrle"]

In [29]:
px.bar(x = series.index[:20], y = series.values[:20])

In [70]:
fig = px.bar(
    df[:50], y = "count", color="gender", text="names",
    color_discrete_sequence=colors, hover_data={"gender": False}
    )

In [75]:
colors =  [
        "rgba(252, 191, 73, 1)", #yellow
        "rgba(214, 40, 40, 1)",  #red
        "rgba(129, 15, 128, 1)", #purple
        "rgba(22, 88, 177, 1)", #blue
        "rgba(6,85,24,1)", #green
        "rgba(1,43, 61, 1)" #dark
        ]
fig.update_layout(
        paper_bgcolor = "#f4f4f4",
        font = {"color":"#181818", "family" : "futura", "size": 13 },
        showlegend = True,
        plot_bgcolor = "#f4f4f4",
        margin = {"t": 0, "b": 0, "l": 0, "r": 0},
        xaxis_title = "Jahr",
        yaxis_title = "Erwähnungen",
        hovermode = "x unified", 
        hoverlabel = {"bgcolor": "white"},
        legend = dict(
            bgcolor = "rgba(0,0,0,0)",
            title = "Geschlecht",
            x = 0.9, y = 1,
            bordercolor = "#f4f4f4",
            borderwidth = 1,
        ),)


fig.update_yaxes(ticklabelposition="inside top", title = "Anzahl Autor:innen",
        gridcolor = "lightgrey", nticks = 7, title_standoff = 30, fixedrange = True)
fig.update_xaxes(visible = False,)

fig.update_traces(hovertemplate = "<b> %{text} </b> <br> %{y}")

ValueError: Invalid property specified for object of type plotly.graph_objs.Bar: 'rangeslider'

Did you mean "base"?

    Valid properties:
        alignmentgroup
            Set several traces linked to the same position axis or
            matching axes to the same alignmentgroup. This controls
            whether bars compute their positional range dependently
            or independently.
        base
            Sets where the bar base is drawn (in position axis
            units). In "stack" or "relative" barmode, traces that
            set "base" will be excluded and drawn in "overlay" mode
            instead.
        basesrc
            Sets the source reference on Chart Studio Cloud for
            base .
        cliponaxis
            Determines whether the text nodes are clipped about the
            subplot axes. To show the text nodes above axis lines
            and tick labels, make sure to set `xaxis.layer` and
            `yaxis.layer` to *below traces*.
        cons

In [24]:
api.get_author("AFP")

{'matches': [{'subtitle': 'Ein Unterausschuss soll nach dem Willen von Union und SPD das Parlament stärker in die Krisenbewältigung einbeziehen – ein "Ablenkungsmanöver", kritisieren FDP und Grüne.',
   'uuid': '6fxOeX17bTvMTJOXAyiW2q',
   'title': 'Große Koalition schlägt Corona-Bundestagsgremium vor',
   'href': 'http://www.zeit.de/politik/deutschland/2021-03/bundestag-gremium-begleitung-corona-politik-pandemie',
   'release_date': '2021-03-02T15:13:23Z',
   'uri': 'http://api.zeit.de/content/6fxOeX17bTvMTJOXAyiW2q',
   'supertitle': 'Corona-Politik',
   'teaser_text': 'Ein Unterausschuss soll nach dem Willen von Union und SPD das Parlament stärker in die Krisenbewältigung einbeziehen – ein "Ablenkungsmanöver", kritisieren FDP und Grüne.',
   'teaser_title': 'Große Koalition schlägt Corona-Bundestagsgremium vor'}],
 'uri': 'http://api.zeit.de/author/AFP',
 'value': 'AFP',
 'href': '',
 'limit': 1,
 'offset': 0,
 'found': 1840,
 'type': 'author',
 'id': 'AFP'}